<a href="https://colab.research.google.com/github/yonatan-co/disaster_not_disaster_classification_tensorflow/blob/main/disaster_not_disaster_dataset_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## setup

In [ ]:
!pip install tensorflow_hub

In [ ]:
!wget https://raw.githubusercontent.com/yonatan-co/t_helpers/main/helpers.py

In [ ]:
from helpers import unzip_data, plot_loss_curves, create_tensorboard_callback, make_confusion_matrix, compare_historys

In [ ]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

## data preprocessing

In [ ]:
unzip_data('nlp_getting_started.zip')

In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

train_df.head(), test_df.head()

In [ ]:
train_df = train_df.sample(frac=1, random_state=42)
test_df = train_df.sample(frac=1, random_state=2)


train_df.head()

In [ ]:
import matplotlib.pyplot as plt



labels = ['not a disaster', 'disaster']
values = [train_df.target.value_counts()[0], train_df.target.value_counts()[1]]


fig1, ax1 = plt.subplots()
ax1.pie(values, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')


plt.show()

In [ ]:
import random

rand_index = random.randint(0, len(train_df)-5)

for row in train_df[["text", "target"]][rand_index:rand_index+5].itertuples():
  _, text, target = row
  print(
  f"""
    tweet:
    {text}
    target: {("disaster" if target == 1 else "not a disaster")}
  """)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                train_df["target"].to_numpy(),
                                                                test_size=0.1,
                                                                random_state=43)

## bulding the model: Universal Sentence Encoder transfer learning

#### prepere callbacks

In [ ]:
import tensorflow as tf


checkpoint_path = 'model-checkpoints/cp.ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                          monitor='val_accuracy',
                                                          save_best_only=True,
                                                          save_weights_only=True,
                                                          verbose=0)

#### define and compile the model

In [ ]:
from tensorflow import keras 


import tensorflow_hub as hub

# feature extraction layer: USE from tf hub
feature_extraction_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                         input_shape=[],
                                         dtype=tf.string,
                                         trainable=False,
                                         name="USE")

model = keras.Sequential(name="USE_transfer_learning_model")

model.add(feature_extraction_layer)
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))


# compile the model
model.compile(optimizer=keras.optimizers.Adam(),
              loss="binary_crossentropy",
              metrics=["accuracy"]) 


#### fit the model

In [ ]:
model_history = model.fit(x=train_tweets,
                          y=train_labels,
                          epochs=5,
                          validation_data=(val_tweets, val_labels),
                          callbacks=[checkpoint_callback])

In [ ]:
plot_loss_curves(model_history)

In [ ]:
model.evaluate(val_tweets, val_labels)

In [ ]:
model.save("disaster_not_disaster_USE")

## expirament with the loaded model

In [ ]:
import tensorflow as tf

In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/tensorflow models/disaster_not_disaster_USE")

In [ ]:
# predictions probs

y_prob = loaded_model.predict(val_tweets)

In [ ]:
# predicted labels

y_pred = tf.round(y_prob)

In [ ]:
import pandas as pd

val_df = pd.DataFrame({"tweet": val_tweets,
                         "label": val_labels,
                         "prediction": tf.squeeze(y_pred),
                         "probability": tf.squeeze(y_prob)})

In [ ]:
wrong_preds = val_df[val_df["label"] != val_df["prediction"]].sort_values("probability", ascending=False)

In [ ]:
wrong_preds.head()

# false positives

In [ ]:
wrong_preds.tail()

# false negatives

In [ ]:
test_df 

In [ ]:
test_tweets = test_df["text"]
test_labels = test_df["target"]

test_tweets.shape, test_labels.shape

In [ ]:
loaded_model.evaluate(test_tweets, test_labels)

In [ ]:
test_prob = loaded_model.predict(test_tweets)
test_pred = tf.round(test_prob)

test_prob[:5], test_pred[:5]

In [ ]:
test_pred_df = pd.DataFrame({"tweet": test_tweets,
                         "label": test_labels,
                         "prediction": tf.squeeze(test_pred),
                         "probability": tf.squeeze(test_prob)})

test_df.head()

In [ ]:
wrong_preds = val_df[val_df["label"] != val_df["prediction"]].sort_values("probability", ascending=False)

In [ ]:
wrong_preds.head()

# false positives

In [ ]:
wrong_preds.tail()

# false negatives